Apply CS-CORE to study the co-expression networks of B cells with a COVID single cell dataset.

The steps are the same as the R tutorial https://changsubiostats.github.io/CS-CORE/articles/CSCORE.html. Please refer to the R tutorial for details on each step.

# 1. Load packages and data

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import time
from CSCORE import CSCORE

We use the same COVID single cell data as in [the tutorial of the CS-CORE R package](https://changsubiostats.github.io/CS-CORE/articles/CSCORE.html), which also provides instructions for downloading the dataset.

The original dataset was provided as a Seurat object, and it can be converted to a scanpy object via the following R codes
```{r}
library(Seurat)
library(SeuratDisk)
covid <- readRDS('data/blish_covid.seu.rds')
updated_covid <- SeuratObject::UpdateSeuratObject(covid) # original Seurat object is out-of-date
DefaultAssay(updated_covid) <- 'RNA' # save UMI counts as Raw in AnnData
SaveH5Seurat(updated_covid, filename='updated_covid.h5Seurat')
Convert("updated_covid.h5Seurat", dest = "h5ad")
```

In [2]:
# load single cell data generated by the codes above.
pbmc = sc.read_h5ad('data/updated_covid.h5ad')

# 2. Select cell types and gene sets to study

In [3]:
# focus on B cells
pbmc_B = pbmc[pbmc.obs['cell.type.coarse'] == 'B',:]
pbmc_B

View of AnnData object with n_obs × n_vars = 5022 × 26361
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rps', 'percent.rpl', 'percent.rrna', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.1', 'seurat_clusters', 'singler', 'Admission.level', 'cell.type.fine', 'cell.type.coarse', 'cell.type', 'IFN1', 'HLA1', 'Donor.orig', 'Donor.full', 'Donor', 'Status', 'Sex', 'DPS', 'DTF', 'Admission', 'Ventilated'
    var: 'SCT_features', '_index', 'features'
    obsm: 'X_umap'
    layers: 'SCT'

In [4]:
# scale counts by seq depths
sc.pp.normalize_total(pbmc_B, target_sum=1)
mean_exp = (pbmc_B.X).sum(axis=0).A1

mean_exp_df = pd.DataFrame({'gene': pbmc_B.var.SCT_features, 'mean_expression': mean_exp})
top_genes_df = mean_exp_df.sort_values(by='mean_expression', ascending=False).head(5000)
# obtain indexes for the gene set of interest (top 5000 highly expressed)
top_genes_indices = top_genes_df.index.astype(int).to_numpy()

/projects/chang/miniconda3/envs/cscore_py/lib/python3.11/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


# 3. Run CS-CORE to infer cell-type-specific co-expression network on the specified gene set

In [5]:
pbmc_B_healthy = pbmc_B[pbmc_B.obs['Status'] == 'Healthy', :]
pbmc_B_healthy

View of AnnData object with n_obs × n_vars = 1994 × 26361
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rps', 'percent.rpl', 'percent.rrna', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.1', 'seurat_clusters', 'singler', 'Admission.level', 'cell.type.fine', 'cell.type.coarse', 'cell.type', 'IFN1', 'HLA1', 'Donor.orig', 'Donor.full', 'Donor', 'Status', 'Sex', 'DPS', 'DTF', 'Admission', 'Ventilated'
    var: 'SCT_features', '_index', 'features'
    obsm: 'X_umap'
    layers: 'SCT'

In [6]:
start = time.time()

res = CSCORE(pbmc_B_healthy, top_genes_indices)

end = time.time()

IRLS converged after 4 iterations.
15 among 5000 genes have negative variance estimates. Their co-expressions with other genes were set to 0.
0.1075% co-expression estimates were greater than 1 and were set to 1.
0.0618% co-expression estimates were greater than 1 and were set to 1.


In [7]:
# elapsed time (seconds)
end - start

13.122948169708252

In [8]:
# three p by p matrices
[res[i].shape for i in range(len(res))]

[(5000, 5000), (5000, 5000), (5000, 5000)]

In [9]:
# estimated co-expressions
res[0][:3,:3]

array([[ 1.        ,  0.6173172 , -0.02418011],
       [ 0.6173172 ,  1.        ,  0.0139189 ],
       [-0.02418011,  0.0139189 ,  1.        ]])

In [10]:
# p values
res[1][:3,:3]

array([[0.        , 0.        , 0.30961328],
       [0.        , 0.        , 0.51643869],
       [0.30961328, 0.51643869, 0.        ]])

In [11]:
# test statistics
res[2][:3,:3]

array([[44.63971912, 26.9073582 , -1.01603381],
       [26.9073582 , 44.63359436,  0.64884482],
       [-1.01603381,  0.64884482, 44.55180167]])

Downstream analyses based on these results are illustrated in the https://changsubiostats.github.io/CS-CORE/articles/CSCORE.html. 